<a href="https://colab.research.google.com/github/alexandregalvaom/DESAFIO/blob/main/controle%20de%20produ%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from datetime import date, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
import ipywidgets as widgets
from IPython.display import display
from google.colab import files

# Cores da empresa
COR_PRIMARIA = '#FFA500'  # Laranja
COR_SECUNDARIA = '#4B4B4B'  # Cinza escuro

# Passo 1: Crie o banco de dados para armazenar os dados de produção diária em uma planilha Excel
def create_database():
    if not os.path.exists('producao.xlsx'):
        df = pd.DataFrame(columns=['Data', 'Seccionadora', 'Fitamento', 'Furação', 'Montagem', 'Embalagem', 'Meta Batida'])
        df.to_excel('producao.xlsx', index=False)

# Passo 2: Função para registrar a produção de um dia
def inserir_producao(data, seccionadora, fitamento, furacao, montagem, embalagem):
    df = pd.read_excel('producao.xlsx')
    meta_seccionadora = 120
    meta_fitamento = 4500
    meta_furacao = 2400
    meta_montagem = 20

    meta_batida = (
        seccionadora >= meta_seccionadora and
        fitamento >= meta_fitamento and
        furacao >= meta_furacao and
        montagem >= meta_montagem
    )
    nova_linha = {'Data': data, 'Seccionadora': seccionadora, 'Fitamento': fitamento, 'Furação': furacao, 'Montagem': montagem, 'Embalagem': embalagem, 'Meta Batida': 'Meta Batida' if meta_batida else 'Meta Não Batida'}
    df = pd.concat([df, pd.DataFrame([nova_linha])], ignore_index=True)
    df.to_excel('producao.xlsx', index=False)

# Passo 3: Função para coletar os dados da produção entre duas datas
def coletar_dados(start_date, end_date, calcular_totais=False):
    df = pd.read_excel('producao.xlsx')
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y', errors='coerce')
    mask = (df['Data'] >= pd.to_datetime(start_date)) & (df['Data'] <= pd.to_datetime(end_date))
    dados_filtrados = df.loc[mask]
    if calcular_totais:
        totais = dados_filtrados[['Seccionadora', 'Fitamento', 'Furação', 'Montagem', 'Embalagem']].sum()
        totais['Data'] = 'Total'
        dados_filtrados = pd.concat([dados_filtrados, pd.DataFrame([totais])], ignore_index=True)
    return dados_filtrados

# Passo 4: Função para gerar gráfico com os dados coletados
def gerar_grafico(dados, periodo, salvar=False):
    fig, ax = plt.subplots(figsize=(10, 6))
    for coluna in ['Seccionadora', 'Fitamento', 'Furação', 'Montagem', 'Embalagem']:
        ax.plot(dados['Data'].dt.strftime('%d/%m/%Y'), dados[coluna].astype(int), label=coluna)

    ax.set_xlabel('Data (formato brasileiro)')
    ax.set_ylabel('Produção (unidades inteiras)')
    ax.set_title(f'Desenvolvimento da Produção ({periodo})')
    ax.legend()
    plt.setp(ax.get_xticklabels(), rotation=45)
    ax.set_ylim(0, dados[['Seccionadora', 'Fitamento', 'Furação', 'Montagem', 'Embalagem']].max().max() + 10)
    plt.tight_layout()
    # Adicionar a marca d'água da empresa
    logo = Image.open('/mnt/data/M BARROS __ marca [02 vertical].png')
    fig.figimage(logo, 10, 10, alpha=0.5, zorder=1)
    if salvar:
        plt.savefig(f'grafico_{periodo}.png', dpi=300)
    plt.show()

# Passo 5: Função principal para realizar a análise da produção
def analisar_producao_customizado(start_date, end_date, salvar_grafico=False):
    dados = coletar_dados(start_date, end_date)

    gerar_grafico(dados, f'{start_date} a {end_date}', salvar=salvar_grafico)

# Passo 6: Função para criar o formulário de entrada de dados no Google Colab
def preencher_dados_diarios():
    start_date_input = widgets.Text(description='Data Início (DD/MM/YYYY):', placeholder='DD/MM/YYYY')
    end_date_input = widgets.Text(description='Data Fim (DD/MM/YYYY):', placeholder='DD/MM/YYYY')
    data_input = widgets.Text(description='Data (DD/MM/YYYY):', placeholder='DD/MM/YYYY')
    seccionadora_input = widgets.IntText(description='Seccionadora:', min=0)
    fitamento_input = widgets.IntText(description='Fitamento:', min=0)
    furacao_input = widgets.IntText(description='Furação:', min=0)
    montagem_input = widgets.IntText(description='Montagem:', min=0)
    embalagem_input = widgets.IntText(description='Embalagem:', min=0)
    botao_enviar = widgets.Button(description='Inserir Produção', style=widgets.ButtonStyle(button_color='black', text_color=COR_PRIMARIA))
    botao_download = widgets.Button(description='Baixar Planilha', style=widgets.ButtonStyle(button_color='black', text_color=COR_PRIMARIA))
    botao_download_grafico = widgets.Button(description='Baixar Gráfico Personalizado', style=widgets.ButtonStyle(button_color='black', text_color=COR_PRIMARIA))
    botao_limpar_dados = widgets.Button(description='Limpar Dados Salvos', style=widgets.ButtonStyle(button_color='black', text_color=COR_PRIMARIA))

    def inserir_dados(b):
        try:
            data_formatada = pd.to_datetime(data_input.value, format='%d/%m/%Y', errors='coerce').strftime('%d/%m/%Y')
        except ValueError:
            print("Erro: Formato de data inválido. Por favor, use DD/MM/YYYY.")
            return

        inserir_producao(data_formatada, seccionadora_input.value, fitamento_input.value,
                        furacao_input.value, montagem_input.value, embalagem_input.value)
        print(f"Produção do dia {data_formatada} inserida com sucesso!")

    def baixar_planilha(b):
        files.download('producao.xlsx')

    def baixar_grafico_customizado(b):
        start_date_dt = pd.to_datetime(start_date_input.value, format='%d/%m/%Y', errors='coerce')
        if pd.isna(start_date_dt):
            print("Erro: Formato de data inválido para a data de início. Por favor, use DD/MM/YYYY.")
            return
        start_date = start_date_dt.strftime('%Y-%m-%d')
        end_date_dt = pd.to_datetime(end_date_input.value, format='%d/%m/%Y', errors='coerce')
        if pd.isna(end_date_dt):
            print("Erro: Formato de data inválido para a data de fim. Por favor, use DD/MM/YYYY.")
            return
        end_date = end_date_dt.strftime('%Y-%m-%d')
        analisar_producao_customizado(start_date, end_date, salvar_grafico=True)
        files.download('grafico_customizado.png')

    def limpar_dados(b):
        df = pd.DataFrame(columns=['Data', 'Seccionadora (cortes/dia)', 'Fitamento (metros/dia)', 'Furação (chapas/dia)', 'Montagem (módulos/dia)', 'Embalagem', 'Meta Batida'])
        df.to_excel('producao.xlsx', index=False)
        print("Todos os dados foram limpos com sucesso!")

    botao_enviar.on_click(inserir_dados)
    botao_enviar.layout.width = '200px'
    botao_download.on_click(baixar_planilha)
    botao_download.layout.width = '200px'
    botao_download_grafico.on_click(baixar_grafico_customizado)
    botao_download_grafico.layout.width = '200px'
    botao_limpar_dados.on_click(limpar_dados)
    botao_limpar_dados.layout.width = '200px'

    display(data_input)
    display(seccionadora_input)
    display(fitamento_input)
    display(furacao_input)
    display(montagem_input)
    display(embalagem_input)
    display(botao_enviar)
    display(start_date_input)
    display(end_date_input)
    display(botao_download_grafico)
    display(botao_download)
    display(botao_limpar_dados)